In [1]:
import findspark
findspark.init('/opt/spark/spark-3.2.1-bin-hadoop3.2')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## Import SparkSession
from pyspark.sql import SparkSession

## Create SparkSession 
spark = SparkSession.builder \
      .master("local") \
      .config("spark.mongodb.input.uri", "mongodb://127.0.0.1") \
      .config("spark.mongodb.output.uri", "mongodb://127.0.0.1") \
      .appName("Spark-MLlib-Titanic") \
      .getOrCreate() 

In [ ]:
from pyspark.sql.types import StructField, IntegerType, StringType, StructType, FloatType
from pyspark.sql.functions import from_json, col

schema = StructType([ \
    StructField("PassengerId",IntegerType(),True), \
    StructField("Survived",IntegerType(),True), \
    StructField("Pclass",IntegerType(),True), \
    StructField("Name", StringType(), True), \
    StructField("Sex", StringType(), True), \
    StructField("Age", FloatType(), True), \
    StructField("SibSp", IntegerType(), True), \
    StructField("Parch", IntegerType(), True), \
    StructField("Ticket", StringType(), True), \
    StructField("Fare", FloatType(), True), \
    StructField("Cabin", StringType(), True), \
    StructField("Embarked", StringType(), True)
  ])

In [ ]:
df = spark.read.csv('hdfs://localhost:9000/data/train.csv', header=True, schema=schema)
df.printSchema()
df.show(truncate=False)

In [ ]:
df = df.toPandas()

id_dict = df.to_dict(orient='records')

In [ ]:
from kafka import KafkaProducer

topic = 'Titanic'
bootstrap_servers = 'localhost:9092'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

In [ ]:
import json

for i in id_dict:
    i = json.dumps(i)
    future = producer.send(topic, bytes(i, encoding='utf-8'))
    print(f'Sending msg: {i}')

    result = future.get(timeout=60)